In [ ]:
!pip install transformers accelerate bitsandbytes --quiet

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer,  pipeline

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)

Device: cuda


In [ ]:
pip install -U bitsandbytes

In [ ]:
# Load Mistral-7B-Instruct-v0.3 in 4-bit mode for GPU memory efficiency
model_name = "mistralai/Mistral-7B-Instruct-v0.3"


tokenizer = AutoTokenizer.from_pretrained(model_name)


model = AutoModelForCausalLM.from_pretrained(
model_name,
device_map="auto", # Automatically maps to GPU
load_in_4bit=True, # Saves GPU memory
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
generator = pipeline(
"text-generation",
model=model,
tokenizer=tokenizer
)

Device set to use cuda:0


In [ ]:
def process_email(email_text):
    prompt = f"""
You are a professional travel email assistant.

STEP 1:
Rewrite the email with correct spelling, grammar, and formatting.

STEP 2:
Extract ONLY factual information explicitly mentioned in the email.
Do NOT guess.
Do NOT infer missing data.

Rules:
- Name = passenger name
- Pax = number of passengers
- City = destination city, not sender city
- Days = total number of days give in the email
- Nights = total number of nights
- Dates must be preserved exactly as written in the email, but in the output JSON, return them in universal ISO 8601 format (`YYYY-MM-DD`) for single dates or as `"YYYY-MM-DD to YYYY-MM-DD"` for ranges.
- Return null if information is unclear
Additional Rules:
- If the passenger name is present anywhere in the email, ALWAYS return it exactly as written.
- Do NOT return null for the name under any circumstance if it is present.
- If passenger name is explicitly mentioned, do NOT return null
- Hotel types = star category only (e.g.,"1-star","2-star", "3-star", "5-star", "7-star")
- Hotel name=
  • Hotel names may appear inside brackets, slashes, commas, or mixed text
  • Ignore locations, areas, descriptors, or extra words
  • Correct obvious spelling and capitalization errors
  • If multiple hotel names are listed, return all valid hotel names as an array.
- Include only sightseeing / attraction activities
- Exclude non-attraction activities (e.g., lunch, dinner, transfers)
- Preserve year if present in date

Return STRICT JSON only.

JSON FORMAT:
{{
  "name": null,
  "pax": null,
  "city": null,
  "date": null,
  "duration": null,
  "days": null,
  "nights": null,
  "hotel_types": null,
  "hotel_name": null,
  "attractions": []
}}

EMAIL:
\"\"\"{email_text}\"\"\"
"""

    output = generator(
        prompt,
        max_new_tokens=512,
        do_sample=False
    )

    return output[0]["generated_text"]

In [ ]:
def process_email_v2(email_text):
    prompt = f"""
You are a professional travel email assistant.

TASK:
1. Rewrite the email with correct spelling, grammar, and formatting.
2. Extract ONLY factual information explicitly mentioned in the email.
   - Do NOT guess
   - Do NOT infer
   - Do NOT normalize beyond spelling/formatting

STRICT RULES:
- Name = passenger name exactly as written
- If a passenger name appears anywhere, it MUST be returned (never null)
- Pax = number of passengers
- City = destination city ONLY
- Days = total number of days explicitly stated
- Nights = total number of nights explicitly stated
- Dates:
  • Preserve the original meaning
  • Output ONLY in ISO 8601
  • Single date → YYYY-MM-DD
  • Range → "YYYY-MM-DD to YYYY-MM-DD"
- Hotel types = star category ONLY (e.g.,"1-star","2-star", "3-star", "5-star", "7-star")
- Hotel name:
  • Hotel names may appear inside brackets, slashes, commas, or mixed text
  • Ignore locations, areas, descriptors, or extra words
  • Correct obvious spelling and capitalization errors
  • If multiple hotel names are listed, return all valid hotel names as an array.

- Attractions:
  • INCLUDE only sightseeing places
  • EXCLUDE meals, shopping, transfers, checkout, packages, timings
- Preserve year if present
- If any field is unclear → return null

OUTPUT RULES:
- Return STRICT JSON only
- No markdown
- No explanations
- No extra keys

JSON SCHEMA:
{{
  "name": null,
  "pax": null,
  "city": null,
  "date": null,
  "duration": null,
  "days": null,
  "nights": null,
  "hotel_types": null,
  "hotel_name": [],
  "attractions": []
}}

EMAIL:
{email_text}
"""

    output = generator(
        prompt,
        max_new_tokens=512,
        do_sample=False
    )

    return output[0]["generated_text"]


In [ ]:
email_text = """
Kind attn Sir,
Greetings from arunesh Travel Services !!! Chennai,
Pls give rates for the below

Pax Name    :- Mr. G V Madhavakrishna x 2 pax- 1 DUBLE ROOM
check in       :- 03-07 Feb 26  04nights/05 days
Flight Time   :-MELBOURNE-SIN 03feb   SQ 228    1640/2120
                      SINGAPORE-MAA 07feb  TR 8414   2050/2240
Hotel            :- 1 star Hotel (boss Hotel/Hilton garden/village bugis hotel,Littel                          India AREA)

DAY 01- ARRVL TRANSFER TO HOTEL
DAY 02- CITY TOUR OF SINGAPORE/ GARDEN BY THE BAY/SANTHOSA ISLAND                 5 IN-ONE PAKG
DAY 03- FULL DAY UNIVERSAL STUDIO WITH LUNCH
DAY 04 -JURONG BIRD PARK-MBS SKY PARK -- EVENING NIGHT SAFARI
DAY O5- FULL DAY SHOPPING-1200 HRS CHECK OUT-1630 DEP TO AIR PORT

PLS   REPLY WITH GOOD RATES
"""
result = process_email(email_text)
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



You are a professional travel email assistant.

STEP 1:
Rewrite the email with correct spelling, grammar, and formatting.

STEP 2:
Extract ONLY factual information explicitly mentioned in the email.
Do NOT guess.
Do NOT infer missing data.

Rules:
- Name = passenger name
- Pax = number of passengers
- City = destination city, not sender city
- Days = total number of days give in the email
- Nights = total number of nights
- Dates must be preserved exactly as written in the email, but in the output JSON, return them in universal ISO 8601 format (`YYYY-MM-DD`) for single dates or as `"YYYY-MM-DD to YYYY-MM-DD"` for ranges.
- Return null if information is unclear
Additional Rules:
- If the passenger name is present anywhere in the email, ALWAYS return it exactly as written.
- Do NOT return null for the name under any circumstance if it is present.
- If passenger name is explicitly mentioned, do NOT return null
- Hotel types = star category only (e.g.,"1-star","2-star", "3-star", "5-star"

{
  "name": "Mr. G V Madhavakrishna",
  "pax": 2,
  "city": "Singapore",
  "date": [
    "2026-02-03",
    "2026-02-07"
  ],
  "duration": "5 days",
  "days": 5,
  "nights": 4,
  "hotel_types": ["7-star"],
  "hotel_name": ["Boss Hotel", "Hilton Garden", "Village Bugis Hotel"],
  "attractions": [
    "City Tour of Singapore",
    "Garden by the Bay",
    "Santhosa Island",
    "Universal Studio",
    "Jurong Bird Park",
    "MBS Sky Park",
    "Night Safari",
    "Shopping"
  ]
}

In [ ]:
email_text = """
Kind Attn sir,
How are you sir,
This is Regarding Group 57pax +child,
Travel date 16-19Jan Malaysia and 19-22 Jan- 06N/07d,
Now the problem is Getting Singapore Visa(Already about for 16Pax visa
Refused and asking for Additional Documents,it is under Process may take time,
pls wait,we will conform the travel dates,after singapore visa
Thank you for Kind co-operation.
"""
result = process_email(email_text)
print(result)